In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

categories = ['sci.space', 'comp.graphics', 'rec.sport.hockey']
dataset = fetch_20newsgroups(subset="train", remove=("headers","footers","quotes"), categories=categories)
print(dataset.data, dataset.target)

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_df=0.8,
    min_df=5
)

tfidf_docs = vectorizer.fit_transform(dataset.data)

In [ ]:
def encode_query(query: str):
    return vectorizer.transform([query])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(query, top_k=5):
    q_vec = encode_query(query)                  # (1, V)
    sims = cosine_similarity(q_vec, tfidf_docs) # (1, N)
    sims = sims[0]                              
    top_idx = np.argsort(-sims)[:top_k]         # اندیس سندهای با شباهت بیشتر

    results = []
    for i in top_idx:
        results.append({
            "index": int(i),
            "similarity": float(sims[i]),
            "text": dataset.data[i][:400] + "..."      # فقط چند خط اول
        })
    return results


In [ ]:
for r in search("space shuttle mission", top_k=3):
    print(r["similarity"])
    print("------")
    print(r["text"])
    print("===============")


In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(tfidf_docs, y)

def pretty_print_results(query, results):
    print(f"\nQUERY: {query}")
    print("=" * 80)
    for r in results:
        print(f"Rank {r['rank']} | Cosine: {r['similarity']:.4f} | Doc idx: {r['doc_idx']}")
        print(f"Class: {r['class_name']}")
        print("-" * 40)
        print(r["text"])
        print("=" * 80)


def search_with_nb(query, top_k=5):
    # 5.1 بردار TF-IDF کوئری
    q_vec = vectorizer.transform([query])   # شکل: (1, V)

    # 5.2 پیش‌بینی توزیع احتمال کلاس‌ها با ناایو بیز
    class_probs = nb_clf.predict_proba(q_vec)[0]   # شکل: (n_classes,)

    # 5.3 انتخاب کلاس با بالاترین احتمال
    best_class_idx = int(np.argmax(class_probs))
    best_class_name = dataset.target[best_class_idx]
    best_class_prob = class_probs[best_class_idx]

    print(f"\n[Naive Bayes] -> Predicted class: {best_class_name}  (P = {best_class_prob:.4f})")

    # 5.4 فیلتر کردن داکیومنت‌های همین کلاس
    mask = (y == best_class_idx)          # بولی آرایه
    docs_in_class = tfidf_docs[mask]      # زیرماتریس TF-IDF همین کلاس
    idx_in_all = np.where(mask)[0]        # ایندکس‌های واقعی در کل دیتاست

    # 5.5 محاسبه‌ی Cosine Similarity بین query و همین کلاس
    sims = cosine_similarity(q_vec, docs_in_class)[0]  # شکل: (num_docs_in_class,)

    # 5.6 انتخاب top-k
    top_local_idx = np.argsort(-sims)[:top_k]   # ایندکس در زیرمجموعه
    results = []
    for rank, local_i in enumerate(top_local_idx, start=1):
        global_idx = int(idx_in_all[local_i])   # ایندکس واقعی در کل دیتاست
        results.append({
            "rank": rank,
            "similarity": float(sims[local_i]),
            "doc_idx": global_idx,
            "class_name": best_class_name,
            "text": dataset.data[global_idx][:400] + "..."
        })

    return results


if __name__ == "__main__":
    # یک تست ساده
    query = "space shuttle mission to repair hubble telescope"
    results = search_with_nb(query, top_k=3)
    pretty_print_results(query, results)